# Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Normalization, Dense, SimpleRNN, LSTM, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load data

In [2]:
# Read in the weather data csv
df = pd.read_csv('data/sydney_weather.csv', parse_dates=['Date'])
# For simplicity, keep necessary columns only
df = df[['Date', 'MedTemp']]
df

,Date,MedTemp
0,2008-02-01,20.95
1,2008-02-02,22.55
2,2008-02-03,23.05
3,2008-02-04,21.50
4,2008-02-05,22.70
...,...,...
3339,2017-06-21,14.10
3340,2017-06-22,14.25
3341,2017-06-23,13.55
3342,2017-06-24,14.70


## Date and time features

Add date and time features (columns) such as year, month, day, season, day of the week, etc.

Example:

In [3]:
df['Day_of_week'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month
df['Season'] = df['Date'].dt.quarter
df.head()

,Date,MedTemp,Day_of_week,Month,Season
0,2008-02-01,20.95,4,2,1
1,2008-02-02,22.55,5,2,1
2,2008-02-03,23.05,6,2,1
3,2008-02-04,21.50,0,2,1
4,2008-02-05,22.70,1,2,1


## Target

Compute the target value we want to predict with a prediction window of 1 day.

Feel free to try other prediction windows, e.g. forecasting 2, 3 or 7 days. Further predictions should have worse results, as the current temperature values are less informative of the far future.

In [4]:
df['target'] = df['MedTemp'].shift(-1).values
df.tail()

,Date,MedTemp,Day_of_week,Month,Season,target
3339,2017-06-21,14.10,2,6,2,14.25
3340,2017-06-22,14.25,3,6,2,13.55
3341,2017-06-23,13.55,4,6,2,14.70
3342,2017-06-24,14.70,5,6,2,13.45
3343,2017-06-25,13.45,6,6,2,NaN


## Clean NaNs

Drop the NaNs that have appeared during the feature engineering process.

In [5]:
df = df.dropna()
df.shape

(3343, 6)

# Prepare train / test

In [6]:
def create_data_for_rnns(X, y, look_back=7):
    dataX, dataY = [], []
    for i in range(len(X) - look_back):
        # Take sequence of length `look_back`
        seqX = X.iloc[i:(i + look_back)].values
        seqY = y.iloc[i:(i + look_back)].values
        dataX.append(seqX)
        dataY.append(seqY)
    return np.array(dataX), np.array(dataY)

In [7]:
X = df.drop(['Date', 'target'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [8]:
X_train.head()

,MedTemp,Day_of_week,Month,Season
0,20.95,4,2,1
1,22.55,5,2,1
2,23.05,6,2,1
3,21.50,0,2,1
4,22.70,1,2,1


In [10]:
X_train_rnn, y_train_rnn = create_data_for_rnns(X_train, y_train, look_back=7)
X_test_rnn, y_test_rnn = create_data_for_rnns(X_test, y_test, look_back=7)

# Sample of first sequence
pd.DataFrame(X_train_rnn[0], columns=X_train.columns)

,MedTemp,Day_of_week,Month,Season
0,20.95,4.0,2.0,1.0
1,22.55,5.0,2.0,1.0
2,23.05,6.0,2.0,1.0
3,21.50,0.0,2.0,1.0
4,22.70,1.0,2.0,1.0
5,23.70,2.0,2.0,1.0
6,22.45,3.0,2.0,1.0


# Models

## Simple

In [105]:
rnn = Sequential([
    Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Normalization(),
    SimpleRNN(16, activation='tanh', return_sequences=True),
    SimpleRNN(16, activation='tanh', return_sequences=False),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])

rnn.compile(optimizer=Adam(learning_rate=0.001), loss='mean_absolute_error')

In [106]:
es = EarlyStopping(patience=5, restore_best_weights=True)
rnn.fit(X_train_rnn, y_train_rnn, epochs=1000, batch_size=64, validation_split=0.2, callbacks=[es])

Epoch 1/1000


34/34 [==============================] - 2s 14ms/step - loss: 16.4281 - val_loss: 14.8137
Epoch 2/1000
34/34 [==============================] - 0s 5ms/step - loss: 13.9234 - val_loss: 12.4439
Epoch 3/1000
34/34 [==============================] - 0s 5ms/step - loss: 11.5676 - val_loss: 10.0347
Epoch 4/1000
34/34 [==============================] - 0s 6ms/step - loss: 8.9485 - val_loss: 7.1723
Epoch 5/1000
34/34 [==============================] - 0s 5ms/step - loss: 6.1489 - val_loss: 4.8076
Epoch 6/1000
34/34 [==============================] - 0s 5ms/step - loss: 4.3552 - val_loss: 3.8908
Epoch 7/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.7375 - val_loss: 3.6841
Epoch 8/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.5856 - val_loss: 3.6416
Epoch 9/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.5463 - val_loss: 3.6337
Epoch 10/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.5247 - val_loss: 3.5936
Epoch

In [107]:
rnn.evaluate(X_train_rnn, y_train_rnn, verbose=0)

1.3812681436538696

In [70]:
rnn2 = Sequential([
    Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Normalization(),
    #SimpleRNN(128, activation='tanh', return_sequences=True),
    SimpleRNN(64, activation='tanh', return_sequences=True),
    SimpleRNN(32, activation='tanh', return_sequences=True),
    SimpleRNN(16, activation='tanh', return_sequences=False),
    Dense(16, activation='relu'),
    #Dense(8, activation='relu'),
    Dense(1, activation='linear')
])

rnn2.compile(optimizer=Adam(learning_rate=0.001), loss='mean_absolute_error')

In [71]:
es = EarlyStopping(patience=5, restore_best_weights=True)
rnn2.fit(X_train_rnn, y_train_rnn, epochs=1000, batch_size=64, validation_split=0.2, callbacks=[es])

Epoch 1/1000
34/34 [==============================] - 2s 13ms/step - loss: 15.1169 - val_loss: 12.9530
Epoch 2/1000
34/34 [==============================] - 0s 4ms/step - loss: 11.2532 - val_loss: 8.7243
Epoch 3/1000
34/34 [==============================] - 0s 4ms/step - loss: 6.8062 - val_loss: 4.6527
Epoch 4/1000
34/34 [==============================] - 0s 4ms/step - loss: 4.0064 - val_loss: 3.6829
Epoch 5/1000
34/34 [==============================] - 0s 3ms/step - loss: 3.5713 - val_loss: 3.6547
Epoch 6/1000
34/34 [==============================] - 0s 4ms/step - loss: 3.5486 - val_loss: 3.6541
Epoch 7/1000
34/34 [==============================] - 0s 3ms/step - loss: 3.5489 - val_loss: 3.6563
Epoch 8/1000
34/34 [==============================] - 0s 4ms/step - loss: 3.5471 - val_loss: 3.6480
Epoch 9/1000
34/34 [==============================] - 0s 3ms/step - loss: 3.5325 - val_loss: 3.5511
Epoch 10/1000
34/34 [==============================] - 0s 3ms/step - loss: 2.4747 - val_loss: 1.

In [72]:
rnn2.evaluate(X_train_rnn, y_train_rnn, verbose=0)

1.3733042478561401

In [73]:
rnn2.evaluate(X_test_rnn, y_test_rnn, verbose=0)

1.4600353240966797

## LSTM

In [108]:
lstm = Sequential([
    Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Normalization(),
    LSTM(16, return_sequences=True),
    LSTM(16, return_sequences=False),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])

lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_absolute_error')

In [109]:
es = EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)
lstm.fit(X_train_rnn, y_train_rnn, epochs=1000, batch_size=64, validation_split=0.2, callbacks=[es])

Epoch 1/1000
34/34 [==============================] - 4s 34ms/step - loss: 17.7294 - val_loss: 16.3399
Epoch 2/1000
34/34 [==============================] - 0s 9ms/step - loss: 15.0299 - val_loss: 12.9354
Epoch 3/1000
34/34 [==============================] - 0s 10ms/step - loss: 11.5083 - val_loss: 9.4693
Epoch 4/1000
34/34 [==============================] - 0s 9ms/step - loss: 8.1724 - val_loss: 6.2877
Epoch 5/1000
34/34 [==============================] - 0s 9ms/step - loss: 5.3364 - val_loss: 4.2669
Epoch 6/1000
34/34 [==============================] - 0s 10ms/step - loss: 3.9475 - val_loss: 3.7378
Epoch 7/1000
34/34 [==============================] - 0s 8ms/step - loss: 3.6114 - val_loss: 3.6505
Epoch 8/1000
34/34 [==============================] - 0s 8ms/step - loss: 3.5535 - val_loss: 3.6537
Epoch 9/1000
34/34 [==============================] - 0s 8ms/step - loss: 3.5491 - val_loss: 3.6552
Epoch 10/1000
34/34 [==============================] - 0s 9ms/step - loss: 3.5492 - val_loss

In [110]:
lstm.evaluate(X_train_rnn, y_train_rnn, verbose=0)

1.369399905204773

In [150]:
lstm2 = Sequential([
    Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Normalization(), 
    LSTM(32, return_sequences=True),   
    LSTM(32, return_sequences=False),
    #Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='linear')
])

lstm2.compile(optimizer=Adam(learning_rate=0.001), loss='mean_absolute_error')

In [151]:
es = EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)
lstm2.fit(X_train_rnn, y_train_rnn, epochs=1000, batch_size=64, validation_split=0.2, callbacks=[es])

Epoch 1/1000
34/34 [==============================] - 4s 23ms/step - loss: 18.5586 - val_loss: 18.0174
Epoch 2/1000
34/34 [==============================] - 0s 5ms/step - loss: 17.5317 - val_loss: 16.3867
Epoch 3/1000
34/34 [==============================] - 0s 5ms/step - loss: 15.6581 - val_loss: 14.1885
Epoch 4/1000
34/34 [==============================] - 0s 5ms/step - loss: 13.0873 - val_loss: 11.2087
Epoch 5/1000
34/34 [==============================] - 0s 5ms/step - loss: 9.6343 - val_loss: 7.1834
Epoch 6/1000
34/34 [==============================] - 0s 5ms/step - loss: 5.5576 - val_loss: 4.0252
Epoch 7/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.7135 - val_loss: 3.6481
Epoch 8/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.5522 - val_loss: 3.6582
Epoch 9/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.5478 - val_loss: 3.6498
Epoch 10/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.5373 - val_los

In [152]:
lstm2.evaluate(X_train_rnn, y_train_rnn, verbose=0)

1.3733494281768799

In [153]:
lstm2.evaluate(X_test_rnn, y_test_rnn, verbose=0)

1.4625808000564575

## GRU

In [111]:
gru = Sequential([
    Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Normalization(),
    GRU(16, return_sequences=True),
    GRU(16, return_sequences=False),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])

gru.compile(optimizer=Adam(learning_rate=0.001), loss='mean_absolute_error')

In [112]:
es = EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)
gru.fit(X_train_rnn, y_train_rnn, epochs=1000, batch_size=64, validation_split=0.2, callbacks=[es])

Epoch 1/1000
34/34 [==============================] - 4s 31ms/step - loss: 18.4429 - val_loss: 17.2931
Epoch 2/1000
34/34 [==============================] - 0s 9ms/step - loss: 16.6333 - val_loss: 15.4137
Epoch 3/1000
34/34 [==============================] - 0s 10ms/step - loss: 14.9724 - val_loss: 14.0426
Epoch 4/1000
34/34 [==============================] - 0s 9ms/step - loss: 13.7381 - val_loss: 12.9070
Epoch 5/1000
34/34 [==============================] - 0s 9ms/step - loss: 12.6394 - val_loss: 11.8149
Epoch 6/1000
34/34 [==============================] - 0s 9ms/step - loss: 11.5113 - val_loss: 10.6515
Epoch 7/1000
34/34 [==============================] - 0s 9ms/step - loss: 10.3244 - val_loss: 9.4367
Epoch 8/1000
34/34 [==============================] - 0s 9ms/step - loss: 9.0822 - val_loss: 8.1656
Epoch 9/1000
34/34 [==============================] - 0s 9ms/step - loss: 7.7921 - val_loss: 6.8733
Epoch 10/1000
34/34 [==============================] - 0s 10ms/step - loss: 6.5317 - 

In [113]:
gru.evaluate(X_train_rnn, y_train_rnn, verbose=0)

1.3671817779541016

In [162]:
gru2 = Sequential([
    Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Normalization(),
    GRU(32, return_sequences=True),
    GRU(32, return_sequences=False),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='linear')
])

gru2.compile(optimizer=Adam(learning_rate=0.001), loss='mean_absolute_error')

In [163]:
es = EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)
gru2.fit(X_train_rnn, y_train_rnn, epochs=1000, batch_size=64, validation_split=0.2, callbacks=[es])

Epoch 1/1000
34/34 [==============================] - 4s 23ms/step - loss: 17.7616 - val_loss: 16.4457
Epoch 2/1000
34/34 [==============================] - 0s 5ms/step - loss: 15.7189 - val_loss: 14.1947
Epoch 3/1000
34/34 [==============================] - 0s 5ms/step - loss: 13.1338 - val_loss: 11.2996
Epoch 4/1000
34/34 [==============================] - 0s 5ms/step - loss: 9.7768 - val_loss: 7.3942
Epoch 5/1000
34/34 [==============================] - 0s 5ms/step - loss: 5.7725 - val_loss: 4.1555
Epoch 6/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.7870 - val_loss: 3.6533
Epoch 7/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.5527 - val_loss: 3.6547
Epoch 8/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.5490 - val_loss: 3.6587
Epoch 9/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.5508 - val_loss: 3.6549
Epoch 10/1000
34/34 [==============================] - 0s 5ms/step - loss: 3.5485 - val_loss:

In [164]:
gru2.evaluate(X_train_rnn, y_train_rnn, verbose=0)

1.3645719289779663

In [165]:
gru2.evaluate(X_test_rnn, y_test_rnn, verbose=0)

1.4455574750900269